In [1]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from api_keys import DarkSkyKey

In [2]:
# read taxi rides file
file = "Resources/Taxi_Trips_Random_Sample.csv"
df = pd.read_csv(file, header=None)
df.columns = ["Trip ID", "Taxi ID","Trip Start Timestamp", "Trip End Timestamp","Trip Seconds",
             "Trip Miles", "Pickup Census Tract", "Dropoff Census Tract", "Pickup Community Area", 
             "Dropoff Community Area", "Fare", "Tips", "Tolls", "Extras", "Trip Total", "Payment Type", 
             "Company", "Pickup Centroid Latitude", "Pickup Centroid Longitude", "Pickup Centroid Location", 
             "Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Dropoff Centroid Location","Community Areas"]
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,11/16/2013 01:45:00 AM,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,09/28/2016 04:45:00 PM,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,12/10/2013 07:15:00 AM,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,02/27/2014 09:00:00 AM,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,10/31/2016 04:15:00 PM,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [3]:
# convert timestamp format for api calls
newdatestart_list = []

for date in df['Trip Start Timestamp']:
    date_obj = datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')
    new_date = date_obj.strftime('%Y-%m-%dT%H:%M:%S')
    newdatestart_list.append(new_date)

df['Trip Start Timestamp'] = newdatestart_list
df.head()


,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,2013-11-16T01:45:00,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,2016-09-28T04:45:00,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,2013-12-10T07:15:00,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,2014-02-27T09:00:00,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,2016-10-31T04:15:00,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [4]:
# Save config information
url = "https://api.darksky.net/forecast/"
weather_df=pd.DataFrame()

# Build query URL and DataFrame
for i in range(len(df)):
    #get info from taxi ride file
    lat = df['Pickup Centroid Latitude'][i]
    lng = df['Pickup Centroid Longitude'][i]
    time = df['Trip Start Timestamp'][i]

    #make API calls with this info
    try:
        query_url = url + DarkSkyKey + "/" + str(lat) +"," + str(lng) + "," + time 
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()['currently'] 
        weather_df.loc[i,'Trip ID'] = df['Trip ID'][i]
        weather_df.loc[i,'time'] = time
        weather_df.loc[i,'summary'] = weather_json['summary']
        weather_df.loc[i,'precipIntensity'] = weather_json['precipIntensity']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'temperature'] = weather_json['temperature']
        weather_df.loc[i,'apparentTemperature'] = weather_json['apparentTemperature']
        weather_df.loc[i,'windSpeed'] = weather_json['windSpeed']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'icon'] = weather_json['icon']
    except:
        print(f"Request no {i} failed, skipping to next line")
        continue

#     if i>50: break    
grouped_weather = weather_df.groupby('icon')['temperature'].count()

grouped_weather

weather_df

Request no 316 failed, skipping to next line
Request no 936 failed, skipping to next line
Request no 1002 failed, skipping to next line
Request no 1003 failed, skipping to next line
Request no 1004 failed, skipping to next line
Request no 1005 failed, skipping to next line
Request no 1006 failed, skipping to next line
Request no 1007 failed, skipping to next line
Request no 1008 failed, skipping to next line
Request no 1009 failed, skipping to next line
Request no 1010 failed, skipping to next line
Request no 1011 failed, skipping to next line
Request no 1012 failed, skipping to next line
Request no 1013 failed, skipping to next line
Request no 1014 failed, skipping to next line
Request no 1015 failed, skipping to next line
Request no 1016 failed, skipping to next line
Request no 1017 failed, skipping to next line
Request no 1018 failed, skipping to next line
Request no 1019 failed, skipping to next line
Request no 1020 failed, skipping to next line
Request no 1021 failed, skipping to 

Request no 1179 failed, skipping to next line
Request no 1180 failed, skipping to next line
Request no 1181 failed, skipping to next line
Request no 1182 failed, skipping to next line
Request no 1183 failed, skipping to next line
Request no 1184 failed, skipping to next line
Request no 1185 failed, skipping to next line
Request no 1186 failed, skipping to next line
Request no 1187 failed, skipping to next line
Request no 1188 failed, skipping to next line
Request no 1189 failed, skipping to next line
Request no 1190 failed, skipping to next line
Request no 1191 failed, skipping to next line
Request no 1192 failed, skipping to next line
Request no 1193 failed, skipping to next line
Request no 1194 failed, skipping to next line
Request no 1195 failed, skipping to next line
Request no 1196 failed, skipping to next line
Request no 1197 failed, skipping to next line
Request no 1198 failed, skipping to next line
Request no 1199 failed, skipping to next line
Request no 1200 failed, skipping t

Request no 1358 failed, skipping to next line
Request no 1359 failed, skipping to next line
Request no 1360 failed, skipping to next line
Request no 1361 failed, skipping to next line
Request no 1362 failed, skipping to next line
Request no 1363 failed, skipping to next line
Request no 1364 failed, skipping to next line
Request no 1365 failed, skipping to next line
Request no 1366 failed, skipping to next line
Request no 1367 failed, skipping to next line
Request no 1368 failed, skipping to next line
Request no 1369 failed, skipping to next line
Request no 1370 failed, skipping to next line
Request no 1371 failed, skipping to next line
Request no 1372 failed, skipping to next line
Request no 1373 failed, skipping to next line
Request no 1374 failed, skipping to next line
Request no 1375 failed, skipping to next line
Request no 1376 failed, skipping to next line
Request no 1377 failed, skipping to next line
Request no 1378 failed, skipping to next line
Request no 1379 failed, skipping t

Request no 1538 failed, skipping to next line
Request no 1539 failed, skipping to next line
Request no 1540 failed, skipping to next line
Request no 1541 failed, skipping to next line
Request no 1542 failed, skipping to next line
Request no 1543 failed, skipping to next line
Request no 1544 failed, skipping to next line
Request no 1545 failed, skipping to next line
Request no 1546 failed, skipping to next line
Request no 1547 failed, skipping to next line
Request no 1548 failed, skipping to next line
Request no 1549 failed, skipping to next line
Request no 1550 failed, skipping to next line
Request no 1551 failed, skipping to next line
Request no 1552 failed, skipping to next line
Request no 1553 failed, skipping to next line
Request no 1554 failed, skipping to next line
Request no 1555 failed, skipping to next line
Request no 1556 failed, skipping to next line
Request no 1557 failed, skipping to next line
Request no 1558 failed, skipping to next line
Request no 1559 failed, skipping t

Request no 1717 failed, skipping to next line
Request no 1718 failed, skipping to next line
Request no 1719 failed, skipping to next line
Request no 1720 failed, skipping to next line
Request no 1721 failed, skipping to next line
Request no 1722 failed, skipping to next line
Request no 1723 failed, skipping to next line
Request no 1724 failed, skipping to next line
Request no 1725 failed, skipping to next line
Request no 1726 failed, skipping to next line
Request no 1727 failed, skipping to next line
Request no 1728 failed, skipping to next line
Request no 1729 failed, skipping to next line
Request no 1730 failed, skipping to next line
Request no 1731 failed, skipping to next line
Request no 1732 failed, skipping to next line
Request no 1733 failed, skipping to next line
Request no 1734 failed, skipping to next line
Request no 1735 failed, skipping to next line
Request no 1736 failed, skipping to next line
Request no 1737 failed, skipping to next line
Request no 1738 failed, skipping t

...

Request no 49447 failed, skipping to next line
Request no 49448 failed, skipping to next line
Request no 49449 failed, skipping to next line
Request no 49450 failed, skipping to next line
Request no 49451 failed, skipping to next line
Request no 49452 failed, skipping to next line
Request no 49453 failed, skipping to next line
Request no 49454 failed, skipping to next line
Request no 49455 failed, skipping to next line
Request no 49456 failed, skipping to next line
Request no 49457 failed, skipping to next line
Request no 49458 failed, skipping to next line
Request no 49459 failed, skipping to next line
Request no 49460 failed, skipping to next line
Request no 49461 failed, skipping to next line
Request no 49462 failed, skipping to next line
Request no 49463 failed, skipping to next line
Request no 49464 failed, skipping to next line
Request no 49465 failed, skipping to next line
Request no 49466 failed, skipping to next line
Request no 49467 failed, skipping to next line
Request no 49

Request no 49623 failed, skipping to next line
Request no 49624 failed, skipping to next line
Request no 49625 failed, skipping to next line
Request no 49626 failed, skipping to next line
Request no 49627 failed, skipping to next line
Request no 49628 failed, skipping to next line
Request no 49629 failed, skipping to next line
Request no 49630 failed, skipping to next line
Request no 49631 failed, skipping to next line
Request no 49632 failed, skipping to next line
Request no 49633 failed, skipping to next line
Request no 49634 failed, skipping to next line
Request no 49635 failed, skipping to next line
Request no 49636 failed, skipping to next line
Request no 49637 failed, skipping to next line
Request no 49638 failed, skipping to next line
Request no 49639 failed, skipping to next line
Request no 49640 failed, skipping to next line
Request no 49641 failed, skipping to next line
Request no 49642 failed, skipping to next line
Request no 49643 failed, skipping to next line
Request no 49

Request no 49798 failed, skipping to next line
Request no 49799 failed, skipping to next line
Request no 49800 failed, skipping to next line
Request no 49801 failed, skipping to next line
Request no 49802 failed, skipping to next line
Request no 49803 failed, skipping to next line
Request no 49804 failed, skipping to next line
Request no 49805 failed, skipping to next line
Request no 49806 failed, skipping to next line
Request no 49807 failed, skipping to next line
Request no 49808 failed, skipping to next line
Request no 49809 failed, skipping to next line
Request no 49810 failed, skipping to next line
Request no 49811 failed, skipping to next line
Request no 49812 failed, skipping to next line
Request no 49813 failed, skipping to next line
Request no 49814 failed, skipping to next line
Request no 49815 failed, skipping to next line
Request no 49816 failed, skipping to next line
Request no 49817 failed, skipping to next line
Request no 49818 failed, skipping to next line
Request no 49

Request no 49974 failed, skipping to next line
Request no 49975 failed, skipping to next line
Request no 49976 failed, skipping to next line
Request no 49977 failed, skipping to next line
Request no 49978 failed, skipping to next line
Request no 49979 failed, skipping to next line
Request no 49980 failed, skipping to next line
Request no 49981 failed, skipping to next line
Request no 49982 failed, skipping to next line
Request no 49983 failed, skipping to next line
Request no 49984 failed, skipping to next line
Request no 49985 failed, skipping to next line
Request no 49986 failed, skipping to next line
Request no 49987 failed, skipping to next line
Request no 49988 failed, skipping to next line
Request no 49989 failed, skipping to next line
Request no 49990 failed, skipping to next line
Request no 49991 failed, skipping to next line
Request no 49992 failed, skipping to next line
Request no 49993 failed, skipping to next line
Request no 49994 failed, skipping to next line
Request no 49

,Trip ID,time,summary,precipIntensity,precipProbability,temperature,apparentTemperature,windSpeed,icon
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,2013-11-16T01:45:00,Partly Cloudy,0.0000,0.00,43.39,40.05,5.59,partly-cloudy-night
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,2016-09-28T04:45:00,Overcast,0.0005,0.14,53.40,53.40,1.25,cloudy
2,c2155aedc41a27673cd498d0fea754a18132d7f9,2013-12-10T07:15:00,Mostly Cloudy,0.0000,0.00,7.20,-4.92,7.53,partly-cloudy-day
3,c1e860354f05150021e1bdeb7696314ca3e15608,2014-02-27T09:00:00,Clear,0.0000,0.00,5.96,-5.93,7.04,clear-day
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,2016-10-31T04:15:00,Clear,0.0000,0.00,43.05,43.05,1.80,clear-night
5,be7313b87f53e20b46020458e7b019c58f80b223,2016-06-15T09:30:00,Partly Cloudy,0.0000,0.00,78.07,79.21,3.38,partly-cloudy-day
6,b08cec8ff3328356d388071edbfcf3f0405edfc6,2013-09-16T06:45:00,Overcast,0.0000,0.00,55.65,55.65,4.99,cloudy
7,c189e7f30cc51e0eb24c7ddbf6d4c9adbdd9f2a7,2014-02-21T11:45:00,Partly Cloudy,0.0000,0.00,29.46,18.98,13.41,partly-cloudy-day
8,bc486809fdc9602698d026865cbfffafccbc0d11,2013-01-19T02:15:00,Clear,0.0000,0.00,39.14,34.19,6.85,clear-night
9,b40db62cca32924d1b26ee28ea29b5dab6ccb242,2014-12-26T10:45:00,Partly Cloudy,0.0000,0.00,44.89,41.36,6.40,partly-cloudy-day


In [9]:
weather_df['summary'].unique()


array(['Partly Cloudy', 'Overcast', 'Mostly Cloudy', 'Clear', 'Drizzle',
       'Light Rain', 'Rain', 'Foggy', 'Humid and Mostly Cloudy', 'Humid',
       'Humid and Partly Cloudy', 'Light Snow'], dtype=object)

In [10]:
try: os.mkdir('Output')
except: pass
weather_df.to_csv("output/weather.csv")
